In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

In [ ]:
%run othello_test_util.ipynb

Der folgende Code dient zum Überprüfen, ob die Heuristik besser ist, wenn für den Gegner die möglichen Züge bestimmt werden, obwohl er nicht an der Reihe ist, oder wenn die durchschnittliche Anzahl an möglichen Zügen in der Spielphase verwendet wird.

In [ ]:
def mobility_heuristic_avg(state):
    x = state.num_pieces
    if state.turn == WHITE:
        return (len(state.possible_moves) - (-0.0113 * x*x + 0.7178 * x + 0.5166)) / 64
    else:
        return ((-0.0113 * x * x + 0.7178 * x + 0.5166) - len(state.possible_moves)) / 64
    
def mobility_heuristic_both(state):
    if state.turn == WHITE:
        return (len(state.possible_moves) - len(get_possible_moves(state, BLACK))) / 64
    else:
        return (len(get_possible_moves(state, WHITE)) - len(state.possible_moves)) / 64

def test_combined_heuristic_both(state):
    if state.num_pieces >= 50:
        return disc_count_heuristic(state)
    mobility = mobility_heuristic_both(state)
    cowthello = cowthello_heuristic(state)
    return 0.5 * mobility + 0.5 * cowthello

def test_combined_heuristic_avg(state):
    if state.num_pieces >= 50:
        return disc_count_heuristic(state)
    mobility = mobility_heuristic_avg(state)
    cowthello = cowthello_heuristic(state)
    return 0.5 * mobility + 0.5 * cowthello

In [ ]:
# actual moves of opponent vs average possible moves
settings = { BLACK: { 'heuristic': test_combined_heuristic_both,
                      'algorithm': alphabeta,
                      'depth': 4,
                      'mode': ai_make_move },
             WHITE: { 'heuristic': test_combined_heuristic_avg,
                      'algorithm': alphabeta,
                      'depth': 4,
                      'mode': ai_make_move }}

get_statistics(20, settings)

Der folgende Code dient dazu, die beste Gewichtung einzelnen Komponenten der kombinierten Heuristik zu bestimmen.

In [ ]:
def heuristic_weighting_interval(num, settings, lower, upper):
    global MOBILITY_PROPORTION
    global MOBILITY_PROPORTION2
    MOBILITY_PROPORTION = (lower + upper) / 2
    MOBILITY_PROPORTION2 = lower
    print("Black mobility:", MOBILITY_PROPORTION, "/ White mobility:", MOBILITY_PROPORTION2)
    [b1,d1,w1] = get_statistics(num, settings)
    MOBILITY_PROPORTION2 = upper
    print("Black mobility:", MOBILITY_PROPORTION, "/ White mobility:", MOBILITY_PROPORTION2)
    [b2,d2,w2] = get_statistics(num, settings)
    
MOBILITY_PROPORTION = 0.5

def test_combined_heuristic_1(state):
    if state.num_pieces >= 50:
        return disc_count_heuristic(state)
    mobility = mobility_heuristic(state)
    cowthello = cowthello_heuristic(state)
    return MOBILITY_PROPORTION * mobility + (1 - MOBILITY_PROPORTION) * cowthello

MOBILITY_PROPORTION2 = 0.5

def test_combined_heuristic_2(state):
    if state.num_pieces >= 50:
        return disc_count_heuristic(state)
    mobility = mobility_heuristic(state)
    cowthello = cowthello_heuristic(state)
    return MOBILITY_PROPORTION2 * mobility + (1 - MOBILITY_PROPORTION2) * cowthello

In [ ]:
settings = { BLACK: { 'heuristic': test_combined_heuristic_1,
                      'algorithm': alphabeta,
                      'depth': 4,
                      'mode': ai_make_move },
             WHITE: { 'heuristic': test_combined_heuristic_2,
                      'algorithm': alphabeta,
                      'depth': 4,
                      'mode': ai_make_move }}

heuristic_weighting_interval(20, settings, 0, 1)

Der folgende Code dient zum Testen, ob der Aufwand, die durchschnittliche Anzahl der möglichen Züge für alle Folgezüge zu berechnen, sich lohnt.

In [ ]:
def get_avg_possible_moves(state):
    moves = []
    for move in state.possible_moves:
        moves.append(len(make_move(copy.deepcopy(state), move).possible_moves))
    return sum(moves) / len(moves)

def mobility_heuristic_exact(state):
    if state.turn == WHITE:
        return (len(state.possible_moves) - get_avg_possible_moves(state)) / 64
    else:
        return (get_avg_possible_moves(state) - len(state.possible_moves)) / 64
    
def mobility_heuristic_approx(state):
    if state.turn == WHITE:
        return (len(state.possible_moves) - len(get_possible_moves(state, BLACK))) / 64
    else:
        return (len(get_possible_moves(state, WHITE)) - len(state.possible_moves)) / 64

def test_combined_heuristic_exact(state):
    if state.num_pieces >= 50:
        return disc_count_heuristic(state)
    mobility = mobility_heuristic_exact(state)
    cowthello = cowthello_heuristic(state)
    return 0.5 * mobility + 0.5 * cowthello

def test_combined_heuristic_approx(state):
    if state.num_pieces >= 50:
        return disc_count_heuristic(state)
    mobility = mobility_heuristic_approx(state)
    cowthello = cowthello_heuristic(state)
    return 0.5 * mobility + 0.5 * cowthello

In [ ]:
settings = { BLACK: { 'heuristic': test_combined_heuristic_exact,
                      'algorithm': alphabeta,
                      'depth': 3,
                      'mode': ai_make_move },
             WHITE: { 'heuristic': test_combined_heuristic_approx,
                      'algorithm': alphabeta,
                      'depth': 3,
                      'mode': ai_make_move }}

get_statistics(20, settings)

In [ ]:
def test_combined_heuristic_pattern(state):
    if state.num_pieces >= 50:
        return disc_count_heuristic(state)
    mobility = mobility_heuristic(state)
    cowthello = cowthello_safe_heuristic(state)
    patterns = pattern_heuristic(state)
    return 0.45 * mobility + 0.45 * cowthello + 0.1 * patterns

In [ ]:
settings = { BLACK: { 'heuristic': combined_heuristic,
                      'algorithm': alphabeta,
                      'depth': 4,
                      'mode': ai_make_move },
             WHITE: { 'heuristic': test_combined_heuristic_pattern,
                      'algorithm': alphabeta,
                      'depth': 4,
                      'mode': ai_make_move }}

get_statistics(50, settings)

Potential mobility

In [ ]:
def test_combined_heuristic(state):
    if state.num_pieces >= 50:
        return disc_count_heuristic(state)
    mobility = mobility_heuristic(state)
    cowthello = cowthello_heuristic(state)
    return 0.5 * mobility + 0.5 * cowthello

def test_combined_heuristic_pot(state):
    if state.num_pieces >= 50:
        return disc_count_heuristic(state)
    mobility = pot_mob_heuristic(state) if state.num_pieces <= 36 else mobility_heuristic(state)
    cowthello = cowthello_heuristic(state)
    return 0.5 * mobility + 0.5 * cowthello

def test_combined_heuristic_pot2(state):
    if state.num_pieces >= 50:
        return disc_count_heuristic(state)
    mobility = (1 - state.num_pieces / 50) * pot_mob_heuristic(state) + (state.num_pieces / 50) *  mobility_heuristic(state)
    cowthello = cowthello_heuristic(state)
    return 0.5 * mobility + 0.5 * cowthello

# Berücksichtigung, dass laut Buro bei 36 Steinen aktuelle und potentielle Mobilität gleich gut sind, vorrausgesetzt die Mobilitäten haben die gleiche Größenordnung
def test_combined_heuristic_pot3(state):
    if state.num_pieces >= 50:
        return disc_count_heuristic(state)
    mobility = (1 - state.num_pieces / 50 + 0.22) * pot_mob_heuristic(state) + (state.num_pieces / 50 - 0.22) *  mobility_heuristic(state)
    cowthello = cowthello_heuristic(state)
    return 0.5 * mobility + 0.5 * cowthello

In [ ]:
settings = { BLACK: { 'heuristic': test_combined_heuristic,
                      'algorithm': alphabeta,
                      'depth': 3,
                      'mode': ai_make_move },
             WHITE: { 'heuristic': test_combined_heuristic_pot2,
                      'algorithm': alphabeta,
                      'depth': 3,
                      'mode': ai_make_move }}

get_statistics(20, settings)